In [14]:
import tensorflow as tf
from tensorflow.keras import layers, Model, Input
from tensorflow.keras.layers import UpSampling2D, Lambda, Cropping2D

from tensorflow.keras import layers
from tensorflow.keras.layers import Resizing, Lambda
from keras.layers import UpSampling2D, ZeroPadding2D, Cropping2D, concatenate
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import CenterCrop

import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import glob
from tf_explain.core.grad_cam import GradCAM

import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.layers import Dense, Dropout, Input, concatenate
from tensorflow.keras.models import Model

from tensorflow.keras.layers import ZeroPadding2D


In [2]:
from tensorflow.keras.layers import Activation
from tensorflow.keras import backend as K
from tensorflow.keras import backend as K
import tensorflow as tf

import gc
import tensorflow as tf


def mish(x):
    return x * K.tanh(K.softplus(x))
  # Use this instead of "mish"






inputs =tf.keras.layers.Input(shape=(224, 224, 3), name="input_tensor")

# First Stage
x = Conv2D(82, (3,3), padding='same')(inputs)
x = BatchNormalization()(x)
x = Activation(mish)(x)

# Second Stage
# Convolution block 1
# one
x = Conv2D(64, (3,3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation(mish)(x)  
# two
x = Conv2D(64, (3,3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation(mish)(x)
# three
x = Conv2D(64, (3,3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation(mish)(x)
# max 1
x = MaxPooling2D(pool_size=(2,2))(x)

# Convolution block 2
# one
x = Conv2D(128, (3,3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation(mish)(x)  
# two
x = Conv2D(128, (3,3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation(mish)(x)
# three
x = Conv2D(128, (3,3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation(mish)(x)
# max 1
x = MaxPooling2D(pool_size=(2,2))(x)

# Convolution block 3
# one
x = Conv2D(128, (3,3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation(mish)(x)  
# two
x = Conv2D(128, (3,3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation(mish)(x)
# three
x = Conv2D(128, (3,3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation(mish)(x)
# max 1
x = MaxPooling2D(pool_size=(2,2))(x)

# Convolution block 4
# one
x = Conv2D(192, (3,3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation(mish)(x)  
# two
x = Conv2D(192, (3,3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation(mish)(x)
# three
x = Conv2D(192, (3,3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation(mish)(x)
# max 1
x = MaxPooling2D(pool_size=(2,2))(x)

# Convolution block 5
# one
x = Conv2D(192, (3,3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation(mish)(x)  
# two
x = Conv2D(192, (3,3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation(mish)(x)
# three
x = Conv2D(192, (3,3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation(mish)(x)
# max 1
x = MaxPooling2D(pool_size=(2,2))(x)
skip5 = x

# Convolution block 6
# one
x = Conv2D(256, (3,3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation(mish)(x)  
# two
x = Conv2D(256, (3,3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation(mish)(x)
# three
x = Conv2D(256, (3,3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation(mish)(x)
# max 1
x = MaxPooling2D(pool_size=(2,2))(x)
skip6 = x

# Convolution block 7
# one
x = Conv2D(256, (3,3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation(mish)(x)  
# two
x = Conv2D(256, (3,3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation(mish)(x)
# three
x = Conv2D(256, (3,3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation(mish)(x)
# max 1
x = MaxPooling2D(pool_size=(2,2))(x)
skip7 = x
# Ensure skip5_up is (7, 7, 192) – No change needed

# Assuming skip5, skip6, and skip7 are your input tensors

# Ensure all tensors are resized to (7, 7)
# skip5_up = UpSampling2D(size=(1, 1), interpolation='bilinear')(skip5)# Already (7x7)

skip6_up = UpSampling2D(size=(2, 2), interpolation='bilinear')(skip6)  
skip6_up = CenterCrop(7, 7)(skip6_up)  # Ensures exact (7, 7) shape

skip7_up = UpSampling2D(size=(4, 4), interpolation='bilinear')(skip7)  
skip7_up = CenterCrop(7, 7)(skip7_up)  # Ensures exact (7, 7) shape


# Now all tensors have shape (7, 7) and can be concatenated
upSampled_layer = concatenate([skip5, skip6_up, skip7_up])

# Third Stage 
gap_output = GlobalAveragePooling2D()(upSampled_layer)
# Desce Block - 1 (1024)
dense1 = Dense(1024)(gap_output)
dense1_mish = Activation(mish)(dense1)  # Mish activation function
dense1_dropOut = Dropout(0.5)(dense1_mish)  # Dropout for regularization
# Dence Block - 2 (128)
dense2 = Dense(128)(dense1_dropOut)
dense2_mish = Activation(mish)(dense2)  # Mish activation function
dense2_dropOut = Dropout(0.5)(dense2_mish)  # Dropout for regularization
# Dence Block - 3 (128)
dense3 = Dense(128)(dense2_dropOut)
dense3_mish = Activation(mish)(dense3)  # Mish activation function
dense3_dropOut = Dropout(0.5)(dense3_mish)  # Dropout for regularization
# Dence Block - 4 (128)
dense4 = Dense(128)(dense3_dropOut)
dense4_mish = Activation(mish)(dense4)  # Mish activation function
dense4_dropOut = Dropout(0.5)(dense4_mish)  # Dropout for regularization

# Output Layer (3 classes: Normal, Benign, Malignant)
output_layer = Dense(3, activation='softmax')(dense4_dropOut)
model = tf.keras.Model(inputs=inputs, outputs=output_layer)
model.summary()



Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_tensor (InputLayer)     │ (None, 224, 224, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d (Conv2D)               │ (None, 224, 224, 82)      │           2,296 │ input_tensor[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization           │ (None, 224, 224, 82)      │             328 │ conv2d[0][0]               │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation (Activation)       │ (None, 224, 224, 82)      │               0 │ batch_normalization[0][0]  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_1 (Conv2D)             │ (None, 224, 224, 64)      │          47,296 │ activation[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_1         │ (None, 224, 224, 64)      │             256 │ conv2d_1[0][0]             │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_1 (Activation)     │ (None, 224, 224, 64)      │               0 │ batch_normalization_1[0][… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_2 (Conv2D)             │ (None, 224, 224, 64)      │          36,928 │ activation_1[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_2         │ (None, 224, 224, 64)      │             256 │ conv2d_2[0][0]             │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_2 (Activation)     │ (None, 224, 224, 64)      │               0 │ batch_normalization_2[0][… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_3 (Conv2D)             │ (None, 224, 224, 64)      │          36,928 │ activation_2[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_3         │ (None, 224, 224, 64)      │             256 │ conv2d_3[0][0]             │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_3 (Activation)     │ (None, 224, 224, 64)      │               0 │ batch_normalization_3[0][… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d (MaxPooling2D)  │ (None, 112, 112, 64)      │               0 │ activation_3[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_4 (Conv2D)             │ (None, 112, 112, 128)     │          73,856 │ max_pooling2d[0][0]        │
├───────────────────────────────┼───────────────────────────┼───────────────

 Total params: 7,110,915 (27.13 MB)

 Trainable params: 7,103,455 (27.10 MB)

 Non-trainable params: 7,460 (29.14 KB)

In [4]:
import tensorflow as tf
import os
import cv2
import numpy as np

def load_image(image_path, target_size=(224, 224)):
    # Load and resize image
    image = cv2.imread(image_path)

    if image is None:
        raise FileNotFoundError(f"Image not found: {image_path}")

    image = cv2.resize(image, target_size) / 255.0  # Normalize to [0, 1]
    return image

def build_dataset(image_dir, label, target_size=(224, 224), num_classes=3):
    images, labels = [], []

    for image_file in os.listdir(image_dir):
        image_path = os.path.join(image_dir, image_file)

        # Load image
        image = load_image(image_path, target_size)
        images.append(image)

        # Convert integer label to one-hot encoded vector
        one_hot_label = tf.keras.utils.to_categorical(label, num_classes)
        labels.append(one_hot_label)

    print(f"Loaded {len(images)} images from {image_dir}")
    return np.array(images), np.array(labels)

def prepare_dataset(base_dir, target_size=(224, 224)):
    categories = ['Normal', 'Benign', 'Malignant']
    image_data, labels = [], []

    for idx, category in enumerate(categories):
        image_dir = os.path.join(base_dir, category, 'Images')
        images, label = build_dataset(image_dir, idx, target_size)

        image_data.append(images)
        labels.append(label)

    images = np.concatenate(image_data)
    labels = np.concatenate(labels)
    
    return images, labels

def create_dataset(image_dir, batch_size=8, target_size=(224, 224)):
    images, labels = prepare_dataset(image_dir, target_size)

    # Create TensorFlow dataset
    dataset = tf.data.Dataset.from_tensor_slices((images, labels))
    dataset = dataset.shuffle(buffer_size=1000).batch(batch_size).prefetch(tf.data.AUTOTUNE)

    return dataset

# Paths to your datasets
train_dir = r"C:\Users\21nn1\OneDrive\Desktop\codee\train_test_validation\train"
val_dir = r"C:\Users\21nn1\OneDrive\Desktop\codee\train_test_validation\val"

batch_size = 8
train_dataset = create_dataset(train_dir, batch_size)
val_dataset = create_dataset(val_dir, batch_size)


Loaded 930 images from C:\Users\21nn1\OneDrive\Desktop\codee\train_test_validation\train\Normal\Images
Loaded 1036 images from C:\Users\21nn1\OneDrive\Desktop\codee\train_test_validation\train\Benign\Images
Loaded 510 images from C:\Users\21nn1\OneDrive\Desktop\codee\train_test_validation\train\Malignant\Images
Loaded 297 images from C:\Users\21nn1\OneDrive\Desktop\codee\train_test_validation\val\Normal\Images
Loaded 319 images from C:\Users\21nn1\OneDrive\Desktop\codee\train_test_validation\val\Benign\Images
Loaded 150 images from C:\Users\21nn1\OneDrive\Desktop\codee\train_test_validation\val\Malignant\Images


In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


# Ensure you cleared the previous model session
tf.keras.backend.clear_session()

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])# Callbacks: Save the best model and apply early stopping
callbacks = [
    ModelCheckpoint('bci_net_best_model.keras', save_best_only=True, monitor='val_accuracy'),
    EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
]

# Train the model (100 epochs with early stopping)
history = model.fit(
    train_dataset,
    epochs=100,
    validation_data=val_dataset,
    callbacks=callbacks
)



Epoch 1/100
310/310 ━━━━━━━━━━━━━━━━━━━━ 1724s 5s/step - accuracy: 0.4748 - loss: 1.1569 - val_accuracy: 0.1958 - val_loss: 1.1502
Epoch 2/100
310/310 ━━━━━━━━━━━━━━━━━━━━ 1627s 5s/step - accuracy: 0.5066 - loss: 1.0552 - val_accuracy: 0.3068 - val_loss: 1.0931
Epoch 3/100
310/310 ━━━━━━━━━━━━━━━━━━━━ 1715s 6s/step - accuracy: 0.5031 - loss: 1.0366 - val_accuracy: 0.5274 - val_loss: 1.0483
Epoch 4/100
310/310 ━━━━━━━━━━━━━━━━━━━━ 1889s 6s/step - accuracy: 0.5121 - loss: 1.0044 - val_accuracy: 0.4060 - val_loss: 1.0460
Epoch 5/100
310/310 ━━━━━━━━━━━━━━━━━━━━ 2816s 9s/step - accuracy: 0.5688 - loss: 0.9774 - val_accuracy: 0.5339 - val_loss: 0.9971
Epoch 6/100
310/310 ━━━━━━━━━━━━━━━━━━━━ 16179s 52s/step - accuracy: 0.5543 - loss: 0.9558 - val_accuracy: 0.3655 - val_loss: 1.0413
Epoch 7/100
310/310 ━━━━━━━━━━━━━━━━━━━━ 1649s 5s/step - accuracy: 0.5568 - loss: 0.9505 - val_accuracy: 0.4634 - val_loss: 1.0012
Epoch 8/100
310/310 ━━━━━━━━━━━━━━━━━━━━ 1664s 5s/step - accuracy: 0.5312 - loss

In [6]:
model.save(r"C:\Users\21nn1\OneDrive\Desktop\codee\models\classification1.keras")
print("model saved")

model saved


In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load the trained model
model = tf.keras.models.load_model(r"C:\Users\jeeva\OneDrive\Desktop\new_data\models\classification1.keras")

# Path to your test dataset
test_data_dir = r"C:\Users\jeeva\OneDrive\Desktop\new_data\train_test_validation\test"

# Ensure the test dataset is preprocessed similarly to training
# Update target_size based on your image dimensions (224, 224)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

test_dataset = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(224, 224),  # Updated image size
    batch_size=32,           # Modify batch size if required
    class_mode='categorical',
    shuffle=False
)

# Evaluate the model on the test dataset
evaluation = model.evaluate(test_dataset)
print(f"Test Loss: {evaluation[0]:.4f}")
print(f"Test Accuracy: {evaluation[1] * 100:.2f}%")

# Generate predictions
predictions = model.predict(test_dataset)
predicted_classes = np.argmax(predictions, axis=1)

# Get the true class labels
true_classes = test_dataset.classes

# Classification report
from sklearn.metrics import classification_report
print(classification_report(true_classes, predicted_classes, target_names=test_dataset.class_indices.keys()))


Found 588 images belonging to 3 classes.
19/19 ━━━━━━━━━━━━━━━━━━━━ 81s 4s/step - accuracy: 0.1525 - loss: 7.5233  
Test Loss: 5.8565
Test Accuracy: 16.84%
19/19 ━━━━━━━━━━━━━━━━━━━━ 94s 5s/step 
              precision    recall  f1-score   support

      Benign       0.17      0.16      0.16       258
   Malignant       0.20      0.43      0.27       122
      Normal       0.07      0.02      0.04       208

    accuracy                           0.17       588
   macro avg       0.14      0.21      0.16       588
weighted avg       0.14      0.17      0.14       588



In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image

# Load the trained model
model = tf.keras.models.load_model(r"C:\Users\jeeva\OneDrive\Desktop\new_data\models\classification1.keras")

# Class labels (update these according to your dataset)
class_labels = ['Normal', 'Benign', 'Malignant']

# Function to classify a single image
def classify_image(img_path):
    # Load and preprocess the image
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Rescale as done during training

    # Make prediction
    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction)

    print(f"Predicted Class: {class_labels[predicted_class]}")
    print(f"Confidence: {prediction[0][predicted_class] * 100:.2f}%")

# Example usage
image_path = r"C:\Users\jeeva\OneDrive\Pictures\Screenshots\Screenshot 2025-03-26 121113.png"
classify_image(image_path)


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
Predicted Class: Malignant
Confidence: 97.51%
